# Import libraries

In [4]:
import time
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
import Utils.DataReader as dr
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.Hybrid.HybridRecommender import DifferentLossRecommender
from Utils.confidence_scaling import linear_scaling_confidence

# Create URMs and ICM

In [ ]:
URM_valid = sps.load_npz('./validDefault.npz')

ICM_tr = dr.load_icm("./UrmTries/data_ICM_type_truncated.csv")

URM_all = dr.load_urm("./UrmTries/RatingsBinary.csv")
URM_train = sps.load_npz('./trainDefault.npz')
stacked_URM = sps.vstack([URM_all, ICM_tr.T])
stacked_URM_train = sps.vstack([URM_train, ICM_tr.T])

URM_train_tanh = sps.load_npz("./trainTanH.npz")
stacked_URM_train_tanh = sps.vstack([URM_train_tanh, ICM_tr.T])
URM_all_tanh = dr.load_urm("./UrmTries/RatingsWithTanHCount.csv")
stacked_URM_tanh = sps.vstack([URM_all_tanh, ICM_tr.T])

evaluator_validation = EvaluatorHoldout(URM_valid, cutoff_list=[10])

# Load or fit recommenders

In [19]:
Urm_in_use = URM_all
stacked_Urm_in_use = stacked_URM

loadStr = '-all'

Urm_tanh_in_use = URM_all_tanh
stacked_Urm_tanh_in_use = stacked_URM_tanh

## Recommenders on binary URM
### SLIM ElasticNet
trained on a stacked URM

In [ ]:
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
recommender_SLIMElasticNet = SLIMElasticNetRecommender(stacked_Urm_in_use)
if loadStr is None:
    recommender_SLIMElasticNet.fit(topK= 950, l1_ratio= 0.004999999999999999, alpha= 0.0015746723778813712)
else:
    recommender_SLIMElasticNet.load_model(folder_path= "./ModelsForFinalSub/", file_name= recommender_SLIMElasticNet.RECOMMENDER_NAME+loadStr+ "Stack")
    recommender_SLIMElasticNet.W_sparse = recommender_SLIMElasticNet.W_sparse.get('W_sparse')

### IALS (implicit library)

In [ ]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
recommender_IALS = IALSRecommender(Urm_in_use)
if loadStr is None:
    recommender_IALS.fit(num_factors= 154, epochs= 140, confidence_scaling= 'linear', alpha=  6.28182586673945, epsilon= 10.0, reg= 1e-05)
else:
    recommender_IALS.load_model(folder_path= "./SavedModels/", file_name= recommender_IALS.RECOMMENDER_NAME + loadStr)

### MultVAER

In [ ]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
recommender_Neural = MultVAERecommender(Urm_in_use)

if loadStr is None:
    recommender_Neural.fit(anneal_cap= 0.1063276470142029, batch_size= 128, dropout=0.15215292346427248, epochs= 131, l2_reg= 0.0014502013635276703, learning_rate= 0.00017395673258925492, total_anneal_steps= 389179.15872038255)
else:
    recommender_Neural.load_model(folder_path= "./SavedModels/", file_name= recommender_Neural.RECOMMENDER_NAME + loadStr)

### RP3Beta
RP3 is a hybrid of an RP3Beta recommender trained on item similarity and another trained on user similarity, both trained on a stacked URM

In [1]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
recommender_RP3_item2 = RP3betaRecommender(stacked_Urm_in_use)
recommender_RP3_item2.fit(alpha= 0.995466740022619, beta= 0.3233559977001016, topK=44, implicit=True)

from Recommenders.GraphBased.RP3betaRecommender import UserRp3Recommender
recommender_RP3_user = UserRp3Recommender(stacked_Urm_in_use)
recommender_RP3_user.fit(topK= 230, alpha= 0.09153459721768885, beta= 0.5796466701328884, normalize_similarity= True)

recommender_Rp3It_RP3Us = DifferentLossRecommender(
    URM_train= stacked_Urm_in_use,
    Recommenders = [recommender_RP3_item2, recommender_RP3_user]
)
recommender_Rp3It_RP3Us.fit(alpha=41.67923805057402, beta=25.013807484352405)


KeyboardInterrupt



### EASE-R
This recommender has been trained on a weighted stacked URM

In [21]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
recommender_EASE = EASE_R_Recommender(URM_train= Urm_in_use, ICM= ICM_tr)
recommender_EASE.load_model(folder_path= "./ModelsForFinalSub/", file_name= recommender_EASE.RECOMMENDER_NAME+loadStr+"K-Stack")
recommender_EASE.W_sparse = recommender_EASE.W_sparse.get('W_sparse')

EASE_R_Recommender: Loading model from file '/Users/nicolacecere/GitHub/RecSysComp-22/ModelsForFinalSub/EASE_R_Recommender-allK-Stack'


# Creating the binary-trained hybrid recommender

In [22]:
recommender_Big6 = DifferentLossRecommender(
    URM_train= stacked_Urm_in_use,
    Recommenders=[
        recommender_SLIMElasticNet,
        recommender_IALS,
        recommender_Neural,
        recommender_Rp3It_RP3Us,
        recommender_EASE
    ]
)

recommender_Big6.fit(
    coefficients=[
        400,
        41.949135309274496,
        4.638442410093666,
        4.536335391442595,
        26.99003518893163
    ]
)

DifferentLossScoresHybridRecommenderLight: URM Detected 3 ( 0.0%) users with no interactions.


In [ ]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender
recommender_implicitAls = ImplicitALSRecommender(stacked_Urm_in_use)
recommender_implicitAls.fit(iterations= int(83.64347328971459), factors= int(125.3856187337561), regularization= 0.05106206327352681, use_gpu=False, num_threads=0, confidence_scaling=linear_scaling_confidence,**{"alpha":5.579621138255679})

# Recommenders on real-valued URM

### EASE-R

In [ ]:
recommender_EASE_tanh = EASE_R_Recommender(Urm_tanh_in_use)
if loadStr is None:
    recommender_EASE_tanh.fit(topK= 286, l2_norm= 27.29893580116881, normalize_matrix= False)
else:
    recommender_EASE_tanh.load_model(folder_path= "./SavedModels/", file_name= recommender_EASE_tanh.RECOMMENDER_NAME+loadStr+ "Tanh")
    recommender_EASE_tanh.W_sparse = recommender_EASE_tanh.W_sparse.get('W_sparse')

### SLIM-ElasticNet

In [ ]:
recommendere_SlimEl_tanh = SLIMElasticNetRecommender(Urm_tanh_in_use)
if loadStr is None:
    recommendere_SlimEl_tanh.fit(topK= 198, l1_ratio= 0.00024230402084727103, alpha= 0.001)
else:
    recommendere_SlimEl_tanh.load_model(folder_path= "./SavedModels/", file_name= recommendere_SlimEl_tanh.RECOMMENDER_NAME+loadStr+ "Tanh")
    recommendere_SlimEl_tanh.W_sparse = recommendere_SlimEl_tanh.W_sparse.get('W_sparse')

### RP3Beta
Stacked URM

In [ ]:
recommender_RP3_tanh = RP3betaRecommender(stacked_Urm_tanh_in_use)
recommender_RP3_tanh.fit(topK=60, alpha=0.3, beta=0.32670025928668045, normalize_similarity= True)

### IALS

In [ ]:
from Recommenders.MatrixFactorization.ImplicitALSRecommender import ImplicitALSRecommender
recommender_implicitAls_tanh = ImplicitALSRecommender(Urm_tanh_in_use)
recommender_implicitAls_tanh.fit(iterations= int(148.98167124505565), factors= int(143.60013227969296), regularization=  0.03055580674080681,use_gpu=False, num_threads=4, confidence_scaling=linear_scaling_confidence,**{"alpha":29.895542778870325})

### MultVAER

In [ ]:
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
recommender_Neural_tanh = MultVAERecommender(Urm_tanh_in_use)
if loadStr is None:
    recommender_Neural_tanh.fit(anneal_cap= 0.2290474595231197, batch_size= 265, dropout=0.5910820786174827, epochs= int(130.140153041107), l2_reg= 0.0005490814004300171, learning_rate= 0.0001985729957209476, total_anneal_steps= 235725.65374509263)
else:
    recommender_Neural_tanh.load_model(folder_path= "./SavedModels/", file_name= recommender_Neural_tanh.RECOMMENDER_NAME+loadStr+ "Tanh")


# Creating the real-valued-URM-trained hybrid recommender

In [27]:
recommender_Big5_tanh = DifferentLossRecommender(
    URM_train= stacked_Urm_in_use,
    Recommenders=[
        recommender_EASE_tanh,
        recommendere_SlimEl_tanh,
        recommender_RP3_tanh,
        recommender_implicitAls_tanh,
        recommender_Neural_tanh
    ]
)

recommender_Big5_tanh.fit(
    coefficients=[
        50.0,
        62.05028485976309,
        200.0,
        22.021908709960933,
        2.2587858482791896
    ]
)

DifferentLossScoresHybridRecommenderLight: URM Detected 3 ( 0.0%) users with no interactions.


# Create the final recommender

In [29]:
recommender = DifferentLossRecommender(
    URM_train= stacked_Urm_in_use,
    Recommenders=[
        recommender_Big6,
        recommender_implicitAls,
        recommender_Big5_tanh
        ]
)
recommender.fit(
    coefficients=[
        45.99743013190489,
        169.71401712945914,
        60.06685090535372
    ]
)

DifferentLossScoresHybridRecommender: URM Detected 3 ( 0.0%) users with no interactions.


In [18]:
evaluator_validation.evaluateRecommender(recommender)

EvaluatorHoldout: Processed 40666 (100.0%) in 2.47 min. Users per second: 275


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.057596                 0.095976  0.089661  0.028559    0.046434   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.188427  0.092215  0.070138  0.374293      0.230096  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.976867          0.365635    0.976867       0.016234   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10            8.692432                   0.993418             0.031492   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
import time
test_users = pd.read_csv('./DatasetCompetition/data_target_users_test.csv')
user_id = test_users['user_id']
recommendations = []
startTime = time.time()
for user in user_id:
    recommendations.append(recommender.recommend(user, cutoff=10))
    if user%1000 == 0:
        print("elapsed: {}".format(time.time()-startTime))
for index in range(len(recommendations)):
    recommendations[index] = np.array(recommendations[index])

test_users['item_list'] = recommendations
test_users['item_list'] = pd.DataFrame(
    [str(line).strip('[').strip(']').replace("'", "") for line in test_users['item_list']])
test_users.to_csv('FINAL_SUB.csv', index=False)